## Imports

In [1]:
import sys
sys.path.append("..")

In [2]:
from snowflake.snowpark import Session
from snowflake.snowpark.functions import when, col, count
from datetime import date
from helpers import SnowflakeHelper
import json
import os

In [3]:
snowflake_helper = SnowflakeHelper()
snowflake_config = './../helpers/snowflake_config.json'
session = snowflake_helper.create_snowpark_session(snowflake_config)
session.use_schema("SAFEGUARDING_NYC_SCHEMA_BRONZE")

[INFO] No schema passed, using default schema SAFEGUARDING_NYC_SCHEMA_BRONZE for the session
[SUCCESS] Config file loaded successfully!
[SUCCESS] Snowspark Session created successfully on schema SAFEGUARDING_NYC_SCHEMA_BRONZE!


## Extracting Data

In [8]:
NYPD_PRECINCTS_NEIGHBORHOODS = session.table('NYPD_PRECINCTS_NEIGHBORHOODS')

In [10]:
NYPD_PRECINCTS_NEIGHBORHOODS.show()

--------------------------------------------------------------------------------------------------------------------------------------------------------
|"_AIRBYTE_RAW_ID"                     |"_AIRBYTE_EXTRACTED_AT"    |"_AIRBYTE_META"  |"NEIGHBORHOODS"                                     |"PRECINCT"  |
--------------------------------------------------------------------------------------------------------------------------------------------------------
|a92c4bf0-1ea7-4f81-b043-c90bdf4f5df2  |2023-12-06 23:26:35-08:00  |{                |World Trade Center/SOHO/Tribeca/Wall Street.        |1           |
|                                      |                           |  "errors": []   |                                                    |            |
|                                      |                           |}                |                                                    |            |
|bfca2bd3-20c9-4337-9b6a-8bb677bccd2a  |2023-12-06 23:26:35-08:00  |{             

## Pre-processing

### Dropping non-important columns

In [12]:
NYPD_PRECINCTS_NEIGHBORHOODS=NYPD_PRECINCTS_NEIGHBORHOODS.drop(["_AIRBYTE_RAW_ID", "_AIRBYTE_EXTRACTED_AT", "_AIRBYTE_META"])

In [13]:
NYPD_PRECINCTS_NEIGHBORHOODS.show()

-------------------------------------------------------------------
|"NEIGHBORHOODS"                                     |"PRECINCT"  |
-------------------------------------------------------------------
|World Trade Center/SOHO/Tribeca/Wall Street.        |1           |
|Chinatown/Little Italy                              |5           |
|Greenwich Village/West Village                      |6           |
|Williamsburg Bridge/Orchard Street                  |7           |
|East Village                                        |9           |
|Chelsea, Clinton/Hell's Kitchen South/Hudson Yards  |10          |
|Peter Cooper Village/Stuyvesant Town residentia...  |13          |
|Sutton Area/Beekman Place/Kipps Bay/Turtle Bay/...  |17          |
|Madison/Lexington/3rd Avenues                       |19          |
|Lincoln Center for the Performing Arts/the Amer...  |20          |
-------------------------------------------------------------------



## Exporting the processed data to Silver Medallion

In [25]:
table_name = "SAFEGUARDING_NYC_SCHEMA_SILVER.NYPD_PRECINCTS_NEIGHBORHOODS"
NYPD_PRECINCTS_NEIGHBORHOODS.write.saveAsTable(table_name, mode="overwrite")